In [4]:
import pandas as pd
import bs4 as bs
import urllib2
import re
import numpy as np
import uuid
import pandas as pd
from os import walk
import json
import sys
import time
reload(sys)
sys.setdefaultencoding('utf-8')

### combine all csv's and remove duplicates

In [57]:
path = '/home/ahmed/Dropbox/DFCI/14_zoo/scrap-science/pubmed/raw_result'
dfs = []
for (_, _, filenames) in walk(path):
    for filename in filenames:
        fullPath = path+'/'+filename
        print fullPath
        dfs.append( pd.read_csv(fullPath, index_col=False,header=0) )
df = pd.concat(dfs)
# reset index
df = df.reset_index(drop=True)
print df.shape
# all titles to lower, strip, remove periods and remove double spaces
df['Title'] = df.Title.str.lower()
df['Title'] = df.Title.str.strip()
df['Title'] = df.Title.str.replace('.','')
df['Title'] = df.Title.str.replace('  ',' ')
# remove duplicates by title
df = df.drop_duplicates(subset='Title', keep="first")
# reset index
df = df.reset_index(drop=True)
print df.shape
df.head()


,Title,URL,Description,Details,ShortDetails,Resource,Type,Identifiers,Db,EntrezUID,Properties
0,development and assessment of a lysophospholip...,/pubmed/28819110,"Long NP, Lim DK, Mo C, Kim G, Kwon SW.",Sci Rep. 2017 Aug 17;7(1):8552. doi: 10.1038/s...,Sci Rep. 2017,PubMed,citation,PMID:28819110 | PMCID:PMC5561257,pubmed,28819110,create date:2017/08/19 | first author:Long NP
1,omni-polya: a method and tool for accurate rec...,/pubmed/28810905,"Magana-Mora A, Kalkatawi M, Bajic VB.",BMC Genomics. 2017 Aug 15;18(1):620. doi: 10.1...,BMC Genomics. 2017,PubMed,citation,PMID:28810905 | PMCID:PMC5558757,pubmed,28810905,create date:2017/08/16 | first author:Magana-M...
2,vps35-related parkinson disease,/pubmed/28796472,"Deutschländer A, Ross OA, Wszolek ZK.","2017 Aug 10. In: Pagon RA, Adam MP, Ardinger H...",GeneReviews<sup>®</sup>. 1993,PubMed,citation,PMID:28796472,pubmed,28796472,create date:2017/08/11 | first author:Deutschl...
3,inicu - integrated neonatal care unit: capturi...,/pubmed/28748430,"Singh H, Yadav G, Mallaiah R, Joshi P, Joshi V...",J Med Syst. 2017 Aug;41(8):132. doi: 10.1007/s...,J Med Syst. 2017,PubMed,citation,PMID:28748430 | PMCID:PMC5529490,pubmed,28748430,create date:2017/07/28 | first author:Singh H
4,deepbipolar: identifying genomic mutations for...,/pubmed/28600868,"Laksshman S, Bhat RR, Viswanath V, Li X.",Hum Mutat. 2017 Sep;38(9):1217-1224. doi: 10.1...,Hum Mutat. 2017,PubMed,citation,PMID:28600868,pubmed,28600868,create date:2017/06/11 | first author:Laksshman S


In [58]:
# add these columns
df['abstract']=""
df['email']=""
df['keywords']=""
df['fullURL']=""
df['source']=['pubmed' for x in range(df.shape[0])]
df['year']=np.zeros(df.shape[0],dtype=np.int)
df['key']=""

#
df['use']=np.zeros(df.shape[0],dtype=np.int)
#archive
print df.shape
df.head(n=4)

,Title,URL,Description,Details,ShortDetails,Resource,Type,Identifiers,Db,EntrezUID,Properties,abstract,email,keywords,fullURL,source,year,key,use
0,development and assessment of a lysophospholip...,/pubmed/28819110,"Long NP, Lim DK, Mo C, Kim G, Kwon SW.",Sci Rep. 2017 Aug 17;7(1):8552. doi: 10.1038/s...,Sci Rep. 2017,PubMed,citation,PMID:28819110 | PMCID:PMC5561257,pubmed,28819110,create date:2017/08/19 | first author:Long NP,,,,,pubmed,0,,0
1,omni-polya: a method and tool for accurate rec...,/pubmed/28810905,"Magana-Mora A, Kalkatawi M, Bajic VB.",BMC Genomics. 2017 Aug 15;18(1):620. doi: 10.1...,BMC Genomics. 2017,PubMed,citation,PMID:28810905 | PMCID:PMC5558757,pubmed,28810905,create date:2017/08/16 | first author:Magana-M...,,,,,pubmed,0,,0
2,vps35-related parkinson disease,/pubmed/28796472,"Deutschländer A, Ross OA, Wszolek ZK.","2017 Aug 10. In: Pagon RA, Adam MP, Ardinger H...",GeneReviews<sup>®</sup>. 1993,PubMed,citation,PMID:28796472,pubmed,28796472,create date:2017/08/11 | first author:Deutschl...,,,,,pubmed,0,,0
3,inicu - integrated neonatal care unit: capturi...,/pubmed/28748430,"Singh H, Yadav G, Mallaiah R, Joshi P, Joshi V...",J Med Syst. 2017 Aug;41(8):132. doi: 10.1007/s...,J Med Syst. 2017,PubMed,citation,PMID:28748430 | PMCID:PMC5529490,pubmed,28748430,create date:2017/07/28 | first author:Singh H,,,,,pubmed,0,,0


In [67]:
years = range(2014,3000)
yearsToInclude = [ str(x) for x in years]

for i in range(995 , df.shape[0] ):   
    
    # get url
    url = 'https://www.ncbi.nlm.nih.gov/' + df.iloc[i].URL
    try:
        # fetch url
        response = urllib2.urlopen(url)
        # convert to bs
        soup = bs.BeautifulSoup(response,"html")
        
        # 0 set fullURL
        df = df.set_value(i, "fullURL", url )

        # 1 get abstract
        abstract = soup.findAll("div",{"class":"abstr"})
        if len(abstract) == 1:
            df = df.set_value(i, "abstract", str(abstract[0]).replace('"',"'") )

        # 2 get email
        afflist = soup.findAll("div", { "class" : "afflist" })
        if len(afflist) == 1:
            email = re.search(r'[\w\.-]+@[\w\.-]+', str(afflist[0]) )
            if email:
                email = email.group(0)
                if email[-1] == '.':
                    email = email[:-1]
                df = df.set_value(i, "email", email)


        # 3 get keywords
        keywords = soup.findAll("div", { "class" : "keywords" })
        if len(keywords) == 1:
            df = df.set_value(i, "keywords", str(keywords[0].p.text).replace('"',"'") )
            
        print i
        
        # 4 set use
        if any(word in df.iloc[i].ShortDetails  for word in yearsToInclude):
            df = df.set_value(i, "use", 1)
            
        # 5 set year
        df = df.set_value(i, "year", int( df.iloc[i].ShortDetails[-4:] ))
        
        # 6 set key
        df = df.set_value(i, "key", str(uuid.uuid4()) )
            
        time.sleep(2)
        df.to_csv("pubmed_temp.csv")
        
    except urllib2.HTTPError:
        print str(i) + " no http"
        pass

   

## remove use = 0 , to make manual check easier.. (should reset_index but i didnt)

In [3]:
# temp
df = pd.DataFrame.from_csv('/home/ahmed/Dropbox/DFCI/14_zoo/scrap-science/pubmed/pubmed_temp.csv')
print df.shape
df = df [df.use == 1]
print df.shape
df = df [df.year >= 2014]
print df.shape
df.to_csv("pubmed_dirty.csv")

# load final and try to add more emails

In [11]:
df = pd.DataFrame.from_csv('/home/ahmed/Dropbox/DFCI/14_zoo/scrap-science/pubmed/out/pubmed_result.csv')
print df.shape
df.head()

,URL,Description,Details,ShortDetails,Resource,Type,Identifiers,Db,EntrezUID,Properties,abstract,email,keywords,fullURL,source,year,key,use
Title,,,,,,,,,,,,,,,,,,
deepbipolar: identifying genomic mutations for bipolar disorder via deep learning,/pubmed/28600868,"Laksshman S, Bhat RR, Viswanath V, Li X.",Hum Mutat. 2017 Sep;38(9):1217-1224. doi: 10.1...,Hum Mutat. 2017,PubMed,citation,PMID:28600868,pubmed,28600868,create date:2017/06/11 | first author:Laksshman S,<div class='abstr'><h3>Abstract</h3><div class...,NaN,bipolar disorder; convolutional neural network...,https://www.ncbi.nlm.nih.gov//pubmed/28600868,pubmed,2017,c3510786-340b-45d7-9272-eee0c8960971,1
precision medicine for heart failure with preserved ejection fraction: an overview,/pubmed/28585183,Shah SJ.,J Cardiovasc Transl Res. 2017 Jun;10(3):233-24...,J Cardiovasc Transl Res. 2017,PubMed,citation,PMID:28585183 | PMCID:PMC5540576,pubmed,28585183,create date:2017/06/07 | first author:Shah SJ,<div class='abstr'><h3>Abstract</h3><div class...,sanjiv.shah@northwestern.edu,Cluster analysis; Heart failure with preserved...,https://www.ncbi.nlm.nih.gov//pubmed/28585183,pubmed,2017,640584a9-ad0c-46cf-97f9-575c47f80b99,1
"deep learning for healthcare: review, opportunities and challenges",/pubmed/28481991,"Miotto R, Wang F, Wang S, Jiang X, Dudley JT.",Brief Bioinform. 2017 May 6. doi: 10.1093/bib/...,Brief Bioinform. 2017,PubMed,citation,PMID:28481991,pubmed,28481991,create date:2017/05/10 | first author:Miotto R,<div class='abstr'><h3>Abstract</h3><div class...,NaN,biomedical informatics; deep learning; electro...,https://www.ncbi.nlm.nih.gov//pubmed/28481991,pubmed,2017,bae357bb-3bec-43da-8cc4-5095bc4d9bbe,1
hla class i binding prediction via convolutional neural networks,/pubmed/28444127,"Vang YS, Xie X.",Bioinformatics. 2017 Sep 1;33(17):2658-2665. d...,Bioinformatics. 2017,PubMed,citation,PMID:28444127,pubmed,28444127,create date:2017/04/27 | first author:Vang YS,<div class='abstr'><h3>Abstract</h3><div class...,NaN,NaN,https://www.ncbi.nlm.nih.gov//pubmed/28444127,pubmed,2017,de37062a-9cfd-460b-9c90-e94689fefd04,1
deepcpg: accurate prediction of single-cell dna methylation states using deep learning,/pubmed/28395661,"Angermueller C, Lee HJ, Reik W, Stegle O.",Genome Biol. 2017 Apr 11;18(1):67. doi: 10.118...,Genome Biol. 2017,PubMed,citation,PMID:28395661 | PMCID:PMC5387360,pubmed,28395661,create date:2017/04/12 | first author:Angermue...,<div class='abstr'><h3>Abstract</h3><div class...,cangermueller@ebi.ac.uk,Artificial neural network; DNA methylation; De...,https://www.ncbi.nlm.nih.gov//pubmed/28395661,pubmed,2017,bb3fc9dd-5fc2-4cd5-9173-485ab5481c7e,1


In [15]:
counter = 0
for i in range(df.shape[0]):# 
    if pd.isnull(df.iloc[i].email):
        counter+=1
print counter